In [1]:

import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import bs4 as soup

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import requests
import re

In [2]:

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [34]:

token = tokenizer.encode("Hello there",return_tensors='pt')


In [35]:
token

tensor([[  101, 29155, 10768,   102]])

In [20]:
results=model(token)

In [21]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5521, -1.2431, -0.3573,  0.2156,  1.5962]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [23]:
int(torch.argmax(results.logits))+1

5

In [24]:
results.logits

tensor([[-0.5521, -1.2431, -0.3573,  0.2156,  1.5962]],
       grad_fn=<AddmmBackward0>)

In [122]:
from bs4 import BeautifulSoup
url=requests.get("https://www.yelp.com/biz/hard-rock-cafe-san-francisco-5")
soup=BeautifulSoup(url.text,'html.parser')
regex=re.compile('.*comment*.')
results=soup.find_all('p',{'class':regex})

In [123]:
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">The last time I went to a Hard Rock Cafe was years ago in Vegas, and didn't remember what it was like. After coming here, makes sense why it wasn't memorable. It's definitely a tourist spot and I was expecting a more exciting menu. Their menu is pretty limited food wise and pricey for what they serve. All they really serve is burgers and chicken sandwiches with maybe 5 other entrees to pick from. My coworker and I decided to go to Pier 39 after a work conference. We ate at the restaurant around 5:30 on a Thursday and we were seated right away. The place wasn't too busy, one large group and maybe 5 or 6 other tables were full. There was a live DJ playing. <br/><br/>I ordered their Mac and cheese and my friend got a burger. We also got the onion ring tower and I ordered their signature hurricane with the souvenir glass for $5 more and you also get 8 oz more of the beverage. The Mac and cheese was ok, i

In [126]:
reviews=[result.text for result in results]

In [127]:
reviews

["The last time I went to a Hard Rock Cafe was years ago in Vegas, and didn't remember what it was like. After coming here, makes sense why it wasn't memorable. It's definitely a tourist spot and I was expecting a more exciting menu. Their menu is pretty limited food wise and pricey for what they serve. All they really serve is burgers and chicken sandwiches with maybe 5 other entrees to pick from. My coworker and I decided to go to Pier 39 after a work conference. We ate at the restaurant around 5:30 on a Thursday and we were seated right away. The place wasn't too busy, one large group and maybe 5 or 6 other tables were full. There was a live DJ playing. I ordered their Mac and cheese and my friend got a burger. We also got the onion ring tower and I ordered their signature hurricane with the souvenir glass for $5 more and you also get 8 oz more of the beverage. The Mac and cheese was ok, it had a strong red bell pepper flavor to it. The burger was good but the fries were bland. Our 

In [128]:
df=pd.DataFrame(np.array(reviews),columns=['reviews'])

In [129]:
df

,reviews
0,The last time I went to a Hard Rock Cafe was y...
1,"Hello Elizabeth, apologies for your unacceptab..."
2,First I have to agree with Jessica's post. If...
3,"Hello Samson, thankyou for the great review. H..."
4,If you're looking for a great meal with a fun ...
5,"Hello, thank you for the great review and 5 st..."
6,Stopped in for lunch before the Blue Angels sh...
7,"Hello Matthew, thank you for the great feedbac..."
8,exceptionally delicious. We had some chicken w...
9,"Hello Katherine, thank you for the great revie..."


In [130]:
df['reviews'][11]

'Hello Vince. Thank you for the great feedback. Hope to see you again soon! '

In [131]:
def sentiment(str):
    token = tokenizer.encode(str,return_tensors='pt')
    result=model(token)
    return int(torch.argmax(result.logits))+1
    
    

In [132]:
sentiment(df['reviews'].iloc[11])

5

In [133]:
df['sentiment']=df['reviews'].apply(lambda x: sentiment(x[:-1]))

In [134]:
df

,reviews,sentiment
0,The last time I went to a Hard Rock Cafe was y...,3
1,"Hello Elizabeth, apologies for your unacceptab...",1
2,First I have to agree with Jessica's post. If...,3
3,"Hello Samson, thankyou for the great review. H...",5
4,If you're looking for a great meal with a fun ...,4
5,"Hello, thank you for the great review and 5 st...",5
6,Stopped in for lunch before the Blue Angels sh...,4
7,"Hello Matthew, thank you for the great feedbac...",3
8,exceptionally delicious. We had some chicken w...,5
9,"Hello Katherine, thank you for the great revie...",5
